In [8]:
import pandas as pd
import numpy as np
import sqlite3

In [9]:
# run transform_and_create_table.ipynb

In [10]:
conn = sqlite3.connect('teste_ailos.db')

base_originacao = pd.read_sql_query("SELECT * FROM base_originacao", conn)
base_carteira = pd.read_sql_query("SELECT * FROM base_carteira", conn)

In [11]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

# RESOLUÇÃO DO EXERCÍCIO EM SQL
Em uma visão mensal (DTBASE), gere uma  tabela consolidada de carteira, com ordenação crescente pela DTBASE, com as seguintes informações:
- datas bases
- quantidade de cooperativas distintas
- quantidade de CPFs distintos
- soma dos saldo devedor
- média dos dias de atraso
- média da quantidade de parcelas
- menos data de originação dos contratos
- contrato com maior número de parcelas
- prazo médio ponderado dos contratos (valor da originação e quantidade de parcelas)
- taxa de contratos com atrasos > 90
- taxa de saldo devedor com atraso > 90 dias

(ou seja, cada linha deverá ter os 11 itens mencionados, e serão 12 linhas na tabela)

In [12]:
%%sql
with all_data as (
    select 
        bc.*
        ,bo.QTPARCELAS
        ,iif(QTDIASATRASO > 90, 1, 0) is_novent_dias_atraso
    from 
        base_carteira bc
    inner join 
        base_originacao bo 
            on bc.CONTRATO = bo.CONTRATO
            and bc.CPF = bo.CPF
            and bc.CONTA = bo.CONTA
            and bc.COOPERATIVA = bo.COOPERATIVA
            and bc.DTCONTRATO = bo.DTCONTRATO
            and bc.VLCONTRATO = bo.VLCONTRATO
    order by 
        date(DTBASE) asc
)

select 
    dt_base
    ,qnt_coop
    ,qnt_cpfs
    ,sum_sddev
    ,avg_dias_atraso
    ,avg_qnt_parcelas
    ,min_dt_contrato
    ,min_dt_contrato_ever           -- visualizar obs no fim do código
    ,(select CONTRATO from base_originacao where QTPARCELAS = max_parcelas order by DTCONTRATO limit 1) as long_contrato
    ,long_contract_ever             -- visualizar obs no fim do código
    ,avg_pond_prazo
    ,percent_contrat_novent_dias
    ,percent_sddev_novent_dias
from 
(
    select 
        date(DTBASE) as dt_base
        ,count(distinct COOPERATIVA) as qnt_coop
        ,count(distinct CPF) as qnt_cpfs
        ,round(sum(SDDEVEDOR), 2) as sum_sddev
        ,round(avg(QTDIASATRASO), 2) as avg_dias_atraso
        ,round(avg(QTPARCELAS), 2) as avg_qnt_parcelas
        ,min(date(DTCONTRATO)) as min_dt_contrato
        ,(select min(date(DTCONTRATO)) from all_data) as min_dt_contrato_ever
    --  ,contrato com maior número de parcelas 
        ,max(QTPARCELAS) as max_parcelas
        ,(select CONTRATO from base_originacao where QTPARCELAS = (select max(QTPARCELAS) from base_originacao) order by DTCONTRATO asc limit 1) as long_contract_ever
        ,round(sum(VLCONTRATO * QTPARCELAS) / sum(VLCONTRATO), 2) as avg_pond_prazo
        ,round(((count(iif(QTDIASATRASO > 90, 1, Null)) / cast(count(distinct CONTRATO) as float))) * 100, 2) as percent_contrat_novent_dias
        ,round((sum(iif(QTDIASATRASO > 90, SDDEVEDOR, 0)) / sum(SDDEVEDOR)) * 100, 2) as percent_sddev_novent_dias
    from
        all_data
    group by 
        date(DTBASE)
    order by 
        date(DTBASE) asc
) x


 * sqlite:///teste_ailos.db
Done.


dt_base,qnt_coop,qnt_cpfs,sum_sddev,avg_dias_atraso,avg_qnt_parcelas,min_dt_contrato,min_dt_contrato_ever,long_contrato,long_contract_ever,avg_pond_prazo,percent_contrat_novent_dias,percent_sddev_novent_dias
2021-01-31,5,3333,13215691.2,22.86,37.0,2013-01-24,2013-01-24,220282,220282,61.9,9.51,3.88
2021-02-28,4,3334,13058845.8,21.79,37.16,2013-01-24,2013-01-24,220282,220282,60.95,9.03,3.69
2021-03-31,5,3599,14389670.4,21.99,37.1,2013-01-24,2013-01-24,220282,220282,63.01,8.86,3.84
2021-04-30,5,3704,15181006.8,21.97,37.23,2013-01-24,2013-01-24,220282,220282,64.66,9.13,3.8
2021-05-31,4,571,2399193.0,28.37,37.71,2014-07-15,2013-01-24,164089,220282,61.57,12.61,5.8
2021-06-30,5,3918,15556938.0,24.62,37.0,2013-01-24,2013-01-24,220282,220282,62.01,10.36,4.38
2021-07-31,5,4011,16769962.8,24.97,37.41,2013-01-24,2013-01-24,220282,220282,69.34,10.6,4.47
2021-08-31,5,4119,17236592.4,26.26,38.34,2013-01-24,2013-01-24,220282,220282,69.31,11.48,4.96
2021-09-30,5,4313,17437828.2,27.24,37.88,2013-01-24,2013-01-24,220282,220282,69.05,11.92,5.14
2021-10-31,5,4395,17903427.6,30.41,38.02,2013-01-24,2013-01-24,220282,220282,69.32,12.72,5.24


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON
Em uma visão mensal (DTBASE), gere uma  tabela consolidada de carteira, com ordenação crescente pela DTBASE, com as seguintes informações:
- datas bases
- quantidade de cooperativas distintas
- quantidade de CPFs distintos
- soma dos saldo devedor
- média dos dias de atraso
- média da quantidade de parcelas
- menos data de originação dos contratos
- contrato com maior número de parcelas
- prazo médio ponderado dos contratos (valor da originação e quantidade de parcelas)
- taxa de contratos com atrasos > 90
- taxa de saldo devedor com atraso > 90 dias

(ou seja, cada linha deverá ter os 11 itens mencionados, e serão 12 linhas na tabela)

In [13]:
# mergeando as duas bases para conseguir fazer as agregações solicitadas
all_data = base_carteira.merge(base_originacao[['CONTRATO', 'CPF', 'CONTA', 'COOPERATIVA', 'DTCONTRATO', 'VLCONTRATO', 'QTPARCELAS']], how='inner', on=['CONTRATO', 'CPF', 'CONTA', 'COOPERATIVA', 'DTCONTRATO', 'VLCONTRATO'])

# fazendo agregações simples 
simple_agg = all_data.groupby('DTBASE').agg(
                                qnt_coop=('COOPERATIVA', 'nunique'),
                                qnt_cpfs=('CPF', 'nunique'),
                                sum_sddev=('SDDEVEDOR', 'sum'),
                                avg_dias_atraso=('QTDIASATRASO', 'mean'),
                                avg_qnt_parcelas=('QTPARCELAS', 'mean'),
                                min_dt_contrato=('DTCONTRATO', 'min'),
                                max_parcela=('QTPARCELAS', 'max')).round(2)

# criando colunas mais avançadas
pond_col = all_data.groupby('DTBASE').apply(lambda x: np.average(x.QTPARCELAS, weights=x.VLCONTRATO)).round(2).rename('avg_pond_prazo')
contracts_novent_dias_col = all_data[all_data.QTDIASATRASO > 90].groupby('DTBASE')['CONTRATO'].count()
sddev_novent_dias_col = all_data[all_data.QTDIASATRASO > 90].groupby('DTBASE')['SDDEVEDOR'].sum()

# gerando tabela consolidada
consol_table = pd.concat([simple_agg, pond_col, contracts_novent_dias_col, sddev_novent_dias_col], axis=1)

In [14]:
# criando as colunas com taxas
consol_table['percent_contrat_novent_dias'] = (consol_table['CONTRATO'] / consol_table['qnt_cpfs']) * 100
consol_table['percent_sddev_novent_dias'] = (consol_table['SDDEVEDOR'] / consol_table['sum_sddev']) * 100

# criando as colunas que buscam dados máximos, sem agregação
## visualizar obs no fim do código
consol_table['min_dt_contrato_ever'] = all_data.DTCONTRATO.min()
consol_table['long_contrato_ever'] = all_data[all_data.QTPARCELAS == all_data.QTPARCELAS.max()].sort_values('DTCONTRATO', ascending=True)[0:1].CONTRATO.squeeze()
consol_table['long_contrato'] = consol_table.max_parcela.apply(lambda x: all_data[all_data.QTPARCELAS == x].sort_values('DTCONTRATO', ascending=True)[0:1].CONTRATO.squeeze())

# trazendo somente as colunas que foram solicitadas
consol_table.reset_index()[[
                            'DTBASE', 
                            'qnt_coop', 
                            'qnt_cpfs', 
                            'sum_sddev', 
                            'avg_dias_atraso', 
                            'avg_qnt_parcelas', 
                            'min_dt_contrato', 
                            'min_dt_contrato_ever', 
                            'long_contrato',
                            'long_contrato_ever', 
                            'avg_pond_prazo', 
                            'percent_contrat_novent_dias', 
                            'percent_sddev_novent_dias']]

,DTBASE,qnt_coop,qnt_cpfs,sum_sddev,avg_dias_atraso,avg_qnt_parcelas,min_dt_contrato,min_dt_contrato_ever,long_contrato,long_contrato_ever,avg_pond_prazo,percent_contrat_novent_dias,percent_sddev_novent_dias
0,2021-01-31 00:00:00,5,3333,13215691.2,22.86,37.00,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,61.90,9.510951,3.878807
1,2021-02-28 00:00:00,4,3334,13058845.8,21.79,37.16,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,60.95,9.028194,3.692398
2,2021-03-31 00:00:00,5,3599,14389670.4,21.99,37.10,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,63.01,8.863573,3.841426
3,2021-04-30 00:00:00,5,3704,15181006.8,21.97,37.23,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,64.66,9.125270,3.800155
4,2021-05-31 00:00:00,4,571,2399193.0,28.37,37.71,2014-07-15 00:00:00,2013-01-24 00:00:00,164089,220282,61.57,12.609457,5.796824
5,2021-06-30 00:00:00,5,3918,15556938.0,24.62,37.00,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,62.01,10.362430,4.379717
6,2021-07-31 00:00:00,5,4011,16769962.8,24.97,37.41,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,69.34,10.595861,4.468425
7,2021-08-31 00:00:00,5,4119,17236592.4,26.26,38.34,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,69.31,11.483370,4.957150
8,2021-09-30 00:00:00,5,4313,17437828.2,27.24,37.88,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,69.05,11.917459,5.138130
9,2021-10-31 00:00:00,5,4395,17903427.6,30.41,38.02,2013-01-24 00:00:00,2013-01-24 00:00:00,220282,220282,69.32,12.718999,5.235757


## Observações:

- Se filtrarmos a base_originação pelo max(QTPARCELAS) vamos ver que existe +1 registros com a mesma quantidade de parcelas, que no caso é a máxima. Então, para trazer o contrato com maior número de parcelas tomei como critério também sua DTCONTRATO. Ou seja, nesse caso o contrato trazido na query e no código python é aquele cujo:</br>

    - QTPARCELAS é a maior de todas (contrato com o maior número de parcelas)</br>
    - DTCONTRATO é a menor de todas (contrato mais antigo)</br>

---
- Como não entendi ao certo se as colunas "menor data de originação dos contratos" e "contrato com maior número de parcelas" se referiam ao contrato mais antigo de todos e o contrato mais longo de todos, respectivamente, independente de filtro ou agregação, trouxe os dois números. Desta forma a legenda é:

    **long_contrato:** é o contrato com o maior número de parcelas dos registros com DTBASE == a data base agregada/filtrada </br>
    **long_contrato_ever:** é o contrato com o maior número de parcelas, independente da DTBASE ou qualquer outro filtro. </br>
    **min_dt_contrato:** é a menor DTCONTRATO dos registros com DTBASE == a data base agregada/filtrada. </br>
    **min_dt_contrato_ever:** é a menor DTCONTRATO da base inteira, independente da DTBASE ou qualquer outro filtro. </br>